# **Mulit-DOC Reading Chatbot using RAG and** **Streamlit**

## Install required libraries

In [23]:

!pip install -q streamlit pymupdf chromadb pyngrok faiss-cpu sentence-transformers mistralai==0.4.2 python-docx streamlit-copy-to-clipboard

## Required API keys and ngrok token

In [24]:
from google.colab import userdata

NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
MISTRAL_API_KEY  = userdata.get('MISTRAL_API_KEY')

if NGROK_AUTH_TOKEN is None:
    print("❗ NGROK_AUTH_TOKEN missing in Colab secrets.")
if MISTRAL_API_KEY is None:
    print("❗ MISTRAL_API_KEY missing in Colab secrets.")

if NGROK_AUTH_TOKEN:
    !ngrok config add-authtoken $NGROK_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## User-interface web-app

In [25]:
%%writefile app.py

import os, tempfile
import streamlit as st
import fitz
from docx import Document
import numpy as np
from typing import List, Dict, Any

from sentence_transformers import SentenceTransformer, util
import chromadb
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

# ---------- Constants ----------
FALLBACK_ANSWER = "I could not find the answer in the uploaded documents."
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "doc_chunks"

# ---------- Page Configuration ----------
st.set_page_config(page_title="Multi-Document Chatbot", layout="wide")
st.title("🤖 Multi-Document Chatbot")

# ---------- Session State ----------
def ensure_state():
    ss = st.session_state
    ss.setdefault("db", None)
    ss.setdefault("chunks", [])
    ss.setdefault("doc_names", [])
    ss.setdefault("history", [])
    ss.setdefault("embedder", SentenceTransformer(EMBED_MODEL))
    ss.setdefault("collection_name", COLLECTION_NAME)
    ss.setdefault("top_k", 4)
    ss.setdefault("chunk_size", 2000)
    ss.setdefault("chunk_overlap", 400)
    return ss

ss = ensure_state()

# ---------- Sidebar ----------
with st.sidebar:
    st.header("1) Upload documents")
    files = st.file_uploader(
        "PDF / DOCX / TXT", type=["pdf", "docx", "txt"],
        accept_multiple_files=True, label_visibility="collapsed"
    )

    st.header("2) Process")
    ss.top_k = st.slider("Top-k retrieved chunks", 1, 12, ss.top_k, 1)
    ss.chunk_size = st.number_input("Chunk size (chars)", 500, 8000, ss.chunk_size, 100)
    ss.chunk_overlap = st.number_input("Chunk overlap (chars)", 0, 4000, ss.chunk_overlap, 50)

    process_btn = st.button("Process Documents", use_container_width=True)
    clear_btn = st.button("Clear Chat History", use_container_width=True)

    st.markdown("---")
    st.caption("API keys via environment:")
    st.code("os.environ['MISTRAL_API_KEY']", language="python")

if clear_btn:
    ss.history = []
    st.rerun()

# ---------- Utilities ----------
def extract_text(uploaded_file, file_type: str) -> str:
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=f".{file_type}") as tmp:
            tmp.write(uploaded_file.getbuffer())
            path = tmp.name
        if file_type == "pdf":
            with fitz.open(path) as doc:
                text = "".join(page.get_text() for page in doc)
        elif file_type == "docx":
            d = Document(path)
            text = "\n".join([p.text for p in d.paragraphs if p.text.strip()])
        elif file_type == "txt":
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()
        else:
            text = ""
        os.remove(path)
        return text
    except Exception as e:
        st.error(f"Error reading {uploaded_file.name}: {e}")
        return ""

def chunk_text(text: str, size: int, overlap: int) -> List[str]:
    step = max(size - overlap, 1)
    return [text[i:i + size] for i in range(0, len(text), step)]

# ---------- Indexing ----------
if files and process_btn:
    with st.spinner("Processing documents..."):
        ss.chunks, ss.doc_names = [], []
        for f in files:
            ext = f.name.split(".")[-1].lower()
            text = extract_text(f, ext)
            if not text:
                continue
            pieces = chunk_text(text, int(ss.chunk_size), int(ss.chunk_overlap))
            ss.chunks.extend(pieces)
            ss.doc_names.extend([f.name] * len(pieces))

        if ss.chunks:
            embedder = SentenceTransformer(EMBED_MODEL)
            ss.embedder = embedder
            embeddings = embedder.encode(ss.chunks, show_progress_bar=True)

            client = chromadb.PersistentClient(path="./chroma_db")
            if ss.collection_name in [c.name for c in client.list_collections()]:
                client.delete_collection(ss.collection_name)
            col = client.create_collection(ss.collection_name, metadata={"hnsw:space": "cosine"})
            col.add(
                documents=ss.chunks,
                embeddings=embeddings.tolist(),
                ids=[str(i) for i in range(len(ss.chunks))],
                metadatas=[{"doc": n} for n in ss.doc_names],
            )
            ss.db = col
            ss.history = []
            st.success(f"Processed {len(files)} document(s).")

# ---------- Chat ----------
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
if ss.db is None:
    st.info("Please upload and process documents to begin.")
elif not MISTRAL_API_KEY:
    st.error("Mistral API key not found in environment. Set MISTRAL_API_KEY.")
else:
    # render previous turns
    for msg in ss.history:
        with st.chat_message(msg["role"]):
            st.markdown(msg["content"])
            if msg["role"] == "assistant" and msg.get("sources"):
                src = ", ".join(sorted(set(msg["sources"])))
                st.caption(f"**Sources:** {src}")

    # input
    if user_q := st.chat_input("Ask a question…"):
        ss.history.append({"role": "user", "content": user_q})
        st.chat_message("user").write(user_q)

        with st.chat_message("assistant"):
            with st.spinner("Thinking…"):
                q_vec = ss.embedder.encode([user_q])
                results = ss.db.query(
                    query_embeddings=q_vec.tolist(),
                    n_results=int(ss.top_k),
                    include=["documents", "metadatas"],
                )
                docs = results["documents"][0]
                metas = results.get("metadatas", [[{}]*ss.top_k])[0]
                sources = [m.get("doc", "unknown") for m in metas]
                context = "\n\n".join(docs)

                client = MistralClient(api_key=MISTRAL_API_KEY)
                messages = [
                    ChatMessage(
                        role="system",
                        content=(
                            "You are a helpful assistant. Answer the user's question "
                            "based ONLY on the provided context. If the answer is not in the "
                            "context, say you could not find the answer.\n\n"
                            f"Context:\n{context}"
                        ),
                    ),
                    ChatMessage(role="user", content=user_q),
                ]
                resp = client.chat(model="mistral-small-latest", messages=messages)
                answer = resp.choices[0].message.content if resp.choices else FALLBACK_ANSWER

                ss.history.append({"role": "assistant", "content": answer, "sources": sources})
                st.markdown(answer)
                if sources:
                    st.caption("**Sources:** " + ", ".join(sorted(set(sources))))
                st.rerun()


Overwriting app.py


## Developer UI interface

In [26]:
%%writefile app_eval.py

# app_eval.py
import os, time, tempfile
import streamlit as st
import fitz
from docx import Document
import numpy as np
import pandas as pd
from collections import Counter
from typing import List, Dict, Any

from sentence_transformers import SentenceTransformer, util
import chromadb
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

# ---------- Constants ----------
FALLBACK_ANSWER = "I could not find the answer in the uploaded documents."
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "doc_chunks"

# ---------- Page ----------
st.set_page_config(page_title="RAG Chatbot Evaluation", layout="wide")
st.title("🔬 RAG Chatbot Evaluation Dashboard")

# ---------- State ----------
def ensure_state():
    ss = st.session_state
    ss.setdefault("db", None)
    ss.setdefault("chunks", [])
    ss.setdefault("doc_names", [])
    ss.setdefault("history", [])
    ss.setdefault("embedder", SentenceTransformer(EMBED_MODEL))
    ss.setdefault("collection_name", COLLECTION_NAME)
    ss.setdefault("top_k", 4)
    ss.setdefault("chunk_size", 2000)
    ss.setdefault("chunk_overlap", 400)
    ss.setdefault("grounding_threshold", 0.55)
    ss.setdefault("sim_threshold", 0.55)
    return ss

ss = ensure_state()

# ---------- Sidebar ----------
with st.sidebar:
    st.subheader("Configuration")
    ss.top_k = st.slider("Top-k", 1, 12, ss.top_k)
    ss.chunk_size = st.number_input("Chunk size", 500, 8000, ss.chunk_size)
    ss.chunk_overlap = st.number_input("Chunk overlap", 0, 4000, ss.chunk_overlap)
    ss.grounding_threshold = st.slider("Groundedness threshold", 0.1, 0.95, ss.grounding_threshold)
    ss.sim_threshold = st.slider("Relevant-chunk sim threshold", 0.1, 0.95, ss.sim_threshold)

    st.markdown("—")
    files = st.file_uploader("Upload Docs", type=["pdf","docx","txt"], accept_multiple_files=True)
    process_btn = st.button("Process Documents", use_container_width=True)

    st.markdown("—")
    bench_csv = st.file_uploader("Optional Benchmark CSV", type=["csv"], help="Columns: question,expected_answer (optional: expected_doc contains filename substring)")
    run_bench = st.button("Run Benchmark (optional)", use_container_width=True)

# ---------- Utils ----------
def extract_text(uploaded_file, file_type: str) -> str:
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=f".{file_type}") as tmp:
            tmp.write(uploaded_file.getbuffer())
            path = tmp.name
        if file_type == "pdf":
            with fitz.open(path) as doc:
                text = "".join(page.get_text() for page in doc)
        elif file_type == "docx":
            d = Document(path)
            text = "\n".join([p.text for p in d.paragraphs if p.text.strip()])
        elif file_type == "txt":
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()
        else:
            text = ""
        os.remove(path)
        return text
    except:
        return ""

def chunk_text(text: str, size: int, overlap: int) -> List[str]:
    step = max(size - overlap, 1)
    return [text[i:i + size] for i in range(0, len(text), step)]

def cosine_sim(a, b) -> float:
    return float(util.cos_sim(a, b).cpu().numpy().squeeze())

def timed(fn, *a, **kw):
    t0 = time.time()
    out = fn(*a, **kw)
    return out, time.time() - t0

def overlap_coverage(answer: str, context: str) -> float:
    """Coverage proxy: word overlap of answer within context."""
    ans_tokens = answer.lower().split()
    ctx_tokens = context.lower().split()
    if not ctx_tokens:
        return 0.0
    overlap = sum((Counter(ans_tokens) & Counter(ctx_tokens)).values())
    return overlap / len(ans_tokens) if ans_tokens else 0.0

def precision_recall_proxies(query_vec, doc_vecs: np.ndarray, sims: List[float], sim_thr: float):
    """Proxy: relevant if sim >= threshold (to the query)."""
    sims_arr = np.array(sims) if len(sims) else np.array([])
    if sims_arr.size == 0:
        return 0.0, 0.0
    precision = float((sims_arr >= sim_thr).sum()) / len(sims_arr)
    recall = float(sims_arr.max())
    return precision, recall

def export_session_metrics_btn():
    if st.button("📥 Download Session Metrics CSV"):
        rows = []
        for msg in [m for m in ss.history if m["role"] == "assistant"]:
            r = {"answer": msg["content"], "is_fallback": msg.get("is_fallback", False)}
            r.update(msg.get("metrics", {}))
            rows.append(r)
        if not rows:
            st.info("No assistant turns yet.")
            return
        df = pd.DataFrame(rows)
        csv = df.to_csv(index=False).encode("utf-8")
        st.download_button("Download CSV", csv, "session_metrics.csv", "text/csv")

# ---------- Indexing ----------
if files and process_btn:
    with st.spinner("Processing..."):
        ss.chunks, ss.doc_names = [], []
        for f in files:
            ext = f.name.split(".")[-1].lower()
            text = extract_text(f, ext)
            if not text:
                continue
            chunks = chunk_text(text, int(ss.chunk_size), int(ss.chunk_overlap))
            ss.chunks.extend(chunks)
            ss.doc_names.extend([f.name]*len(chunks))

        if ss.chunks:
            embeddings = ss.embedder.encode(ss.chunks, show_progress_bar=True)
            client = chromadb.PersistentClient(path="./chroma_db")
            if ss.collection_name in [c.name for c in client.list_collections()]:
                client.delete_collection(ss.collection_name)
            col = client.create_collection(ss.collection_name, metadata={"hnsw:space":"cosine"})
            col.add(
                documents=ss.chunks,
                embeddings=embeddings.tolist(),
                ids=[str(i) for i in range(len(ss.chunks))],
                metadatas=[{"doc": n} for n in ss.doc_names],
            )
            ss.db = col
            ss.history = []
            st.success(f"Processed {len(files)} document(s).")

# ---------- Live Dashboard ----------
def render_dashboard():
    st.subheader("📈 Live Session Performance")
    assistant_turns = [m for m in ss.history if m["role"] == "assistant"]
    turn_count = len([m for m in ss.history if m["role"] == "user"])
    if turn_count == 0:
        st.info("Start chatting below to populate metrics.")
        return

    fallback_count = sum(1 for m in assistant_turns if m.get("is_fallback"))
    feedbacks = [m.get("feedback") for m in assistant_turns if m.get("feedback") is not None]
    satisfaction_rate = (feedbacks.count("positive")/len(feedbacks)*100) if feedbacks else 0.0
    latencies = [m.get("metrics",{}).get("total_latency_s", 0.0) for m in assistant_turns]
    avg_latency = np.mean(latencies) if latencies else 0.0

    col1, col2, col3, col4 = st.columns(4)
    col1.metric("Total Queries", f"{turn_count}")
    col2.metric("Avg. Latency (s)", f"{avg_latency:.2f}")
    col3.metric("Fallback Rate", f"{(fallback_count/turn_count*100):.1f}%")
    col4.metric("User Satisfaction", f"{satisfaction_rate:.1f}%")

render_dashboard()
st.markdown("---")

# ---------- Chat + Metrics ----------
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
if ss.db and MISTRAL_API_KEY:
    for i, msg in enumerate(ss.history):
        with st.chat_message(msg["role"]):
            st.markdown(msg["content"])
            if msg["role"] == "assistant":
                m = msg.get("metrics", {})
                if m:
                    st.markdown("---")
                    c = st.columns(6)
                    c[0].metric("⏱️ Latency", f"{m.get('total_latency_s',0):.2f}s")
                    c[1].metric("🔎 Avg Q–Doc", f"{m.get('avg_q_doc_sim',0):.2f}")
                    c[2].metric("🏅 Max Q–Doc", f"{m.get('max_q_doc_sim',0):.2f}")
                    c[3].metric("Groundedness", f"{m.get('answer_context_sim',0):.2f}")
                    c[4].metric("Relevance", f"{m.get('answer_relevance_sim',0):.2f}")
                    c[5].metric("Coverage", f"{m.get('coverage',0):.2f}")
                    cc = st.columns(3)
                    cc[0].metric("Precision (proxy)", f"{m.get('context_precision_proxy',0):.2f}")
                    cc[1].metric("Recall (proxy)", f"{m.get('context_recall_proxy',0):.2f}")
                    cc[2].metric("Ans Len", f"{m.get('answer_length',0)}")

                if msg.get("hallucination_risk"):
                    st.warning("⚠️ Potential low grounding detected (below threshold).")
                with st.expander("Retrieved chunks & scores"):
                    for j, (doc, sim) in enumerate(zip(msg.get("retrieved_docs",[]), msg.get("retrieved_sims",[])), 1):
                        st.markdown(f"**Top {j} • Sim:** {sim:.3f} • **Source:** `{doc['source']}`")
                        st.code(doc["content"][:1000] + ("..." if len(doc["content"])>1000 else ""), language="markdown")

                fb_cols = st.columns([1,1,8])
                if fb_cols[0].button("👍", key=f"up_{i}", disabled=(msg.get("feedback") is not None)):
                    msg["feedback"] = "positive"; st.rerun()
                if fb_cols[1].button("👎", key=f"down_{i}", disabled=(msg.get("feedback") is not None)):
                    msg["feedback"] = "negative"; st.rerun()

    # input
    if user_q := st.chat_input("Ask a question…"):
        ss.history.append({"role": "user", "content": user_q})
        st.chat_message("user").write(user_q)
        with st.chat_message("assistant"):
            with st.spinner("Scoring response…"):
                # Retrieval
                q_vec, t_emb = timed(ss.embedder.encode, [user_q])
                res, t_ret = timed(
                    ss.db.query,
                    query_embeddings=q_vec.tolist(),
                    n_results=int(ss.top_k),
                    include=["documents", "distances", "metadatas", "embeddings"]
                )
                docs, dist, meta, embs = (
                    res["documents"][0],
                    res.get("distances", [[]])[0],
                    res.get("metadatas", [[]])[0],
                    np.array(res.get("embeddings", [[]])[0], dtype=np.float32) if res.get("embeddings") else np.array([]),
                )
                sims = [1.0 - d for d in dist] if dist else []
                context = "\n\n".join(docs)

                # Generation
                client = MistralClient(api_key=MISTRAL_API_KEY)
                messages = [
                    ChatMessage(role="system", content=(
                        "Answer based ONLY on the context. If not present, say you could not find the answer.\n\n"
                        f"Context:\n{context}"
                    )),
                    ChatMessage(role="user", content=user_q),
                ]
                resp, t_gen = timed(client.chat, model="mistral-small-latest", messages=messages)
                answer = resp.choices[0].message.content if resp.choices else FALLBACK_ANSWER

                # Embeddings for metrics
                ans_vec = ss.embedder.encode([answer])[0]
                ctx_vec = ss.embedder.encode([context])[0] if context.strip() else np.zeros_like(ans_vec)

                if embs is not None and len(embs) > 0:
                    avg_embs_vec = np.mean(np.array(embs), axis=0).astype(np.float32)
                    answer_relevance_sim = cosine_sim(ans_vec, avg_embs_vec)
                else:
                    answer_relevance_sim = 0.0

                avg_q_doc = float(np.mean(sims)) if sims else 0.0
                max_q_doc = float(np.max(sims)) if sims else 0.0
                groundedness = cosine_sim(ans_vec, ctx_vec) if context.strip() else 0.0
                hallucination_score = 1.0 - groundedness
                coverage = overlap_coverage(answer, context)
                ans_len = len(answer.split())

                # Precision/Recall (proxies) on retrieved docs
                context_precision_proxy, context_recall_proxy = precision_recall_proxies(q_vec[0], embs, sims, float(ss.sim_threshold))

                metrics = {
                    "total_latency_s": float(t_emb + t_ret + t_gen),
                    "avg_q_doc_sim": avg_q_doc,
                    "max_q_doc_sim": max_q_doc,
                    "answer_context_sim": groundedness,
                    "answer_relevance_sim": answer_relevance_sim,
                    "hallucination_score": hallucination_score,
                    "coverage": coverage,
                    "answer_length": ans_len,
                    "context_precision_proxy": context_precision_proxy,
                    "context_recall_proxy": context_recall_proxy,
                }

                ss.history.append({
                    "role": "assistant",
                    "content": answer,
                    "metrics": metrics,
                    "is_fallback": (answer == FALLBACK_ANSWER),
                    "feedback": None,
                    "hallucination_risk": (groundedness < float(ss.grounding_threshold)),
                    "retrieved_docs": [{"content": d, "source": m.get("doc")} for d, m in zip(docs, meta)],
                    "retrieved_sims": sims,
                })
                st.markdown(answer)
                st.rerun()
else:
    if ss.db is None:
        st.info("Please upload and process documents.")
    elif not MISTRAL_API_KEY:
        st.error("MISTRAL_API_KEY not found in environment.")

st.markdown("---")
export_session_metrics_btn()

# ---------- Optional: Benchmark runner ----------
if ss.db and MISTRAL_API_KEY and bench_csv is not None and run_bench:
    df = pd.read_csv(bench_csv)
    if "question" not in df.columns:
        st.error("CSV must have a 'question' column.")
    else:
        st.info("Running benchmark… This will iterate through rows and compute metrics.")
        results = []
        client = chromadb.PersistentClient(path="./chroma_db")
        col = client.get_collection(ss.collection_name)

        mistral = MistralClient(api_key=MISTRAL_API_KEY)

        for idx, row in df.iterrows():
            q = str(row["question"])
            gt = str(row.get("expected_answer", "")) if "expected_answer" in df.columns else ""
            expected_doc = str(row.get("expected_doc", "")) if "expected_doc" in df.columns else ""

            # Retrieval
            q_vec = ss.embedder.encode([q])
            res = col.query(
                query_embeddings=q_vec.tolist(),
                n_results=int(ss.top_k),
                include=["documents","distances","metadatas","embeddings"]
            )
            docs = res["documents"][0]
            dist = res.get("distances",[[]])[0]
            meta = res.get("metadatas",[[]])[0]
            sims = [1.0 - d for d in dist] if dist else []
            context = "\n\n".join(docs)

            # Generation
            messages = [
                ChatMessage(role="system", content=(
                    "Answer based ONLY on the context. If not present, say you could not find the answer.\n\n"
                    f"Context:\n{context}"
                )),
                ChatMessage(role="user", content=q),
            ]
            resp = mistral.chat(model="mistral-small-latest", messages=messages)
            ans = resp.choices[0].message.content if resp.choices else FALLBACK_ANSWER

            # Metrics
            ans_vec = ss.embedder.encode([ans])[0]
            ctx_vec = ss.embedder.encode([context])[0] if context.strip() else np.zeros_like(ans_vec)
            groundedness = cosine_sim(ans_vec, ctx_vec) if context.strip() else 0.0

            # Answer relevance vs retrieved embeddings centroid
            embs = np.array(res.get("embeddings",[[]])[0], dtype=np.float32) if res.get("embeddings") else np.array([])
            if embs.size > 0:
                answer_relevance_sim = cosine_sim(ans_vec, np.mean(embs, axis=0).astype(np.float32))
            else:
                answer_relevance_sim = 0.0

            coverage = overlap_coverage(ans, context)
            avg_q_doc = float(np.mean(sims)) if sims else 0.0
            max_q_doc = float(np.max(sims)) if sims else 0.0
            precision_proxy, recall_proxy = precision_recall_proxies(q_vec[0], embs, sims, float(ss.sim_threshold))

            # Optional semantic accuracy vs ground-truth
            if gt.strip():
                gt_vec = ss.embedder.encode([gt])[0]
                semantic_accuracy = (cosine_sim(ans_vec, gt_vec) + 1) / 2  # map [-1,1] -> [0,1]
            else:
                semantic_accuracy = np.nan

            # Optional retrieval recall@k using expected_doc matching
            if expected_doc and meta:
                retrieved_names = [m.get("doc","") for m in meta]
                recall_at_k = 1.0 if any(expected_doc.lower() in (n or "").lower() for n in retrieved_names) else 0.0
            else:
                recall_at_k = np.nan

            results.append({
                "question": q,
                "answer": ans,
                "avg_q_doc_sim": avg_q_doc,
                "max_q_doc_sim": max_q_doc,
                "answer_context_sim": groundedness,
                "answer_relevance_sim": answer_relevance_sim,
                "coverage": coverage,
                "context_precision_proxy": precision_proxy,
                "context_recall_proxy": recall_proxy,
                "semantic_accuracy_vs_gt": semantic_accuracy,
                "retrieval_recall_at_k_label": recall_at_k,
            })

        bench_df = pd.DataFrame(results)
        st.success("Benchmark finished.")
        st.dataframe(bench_df.head(20))
        st.download_button(
            "📥 Download Benchmark Results",
            bench_df.to_csv(index=False).encode("utf-8"),
            "benchmark_results.csv",
            "text/csv"
        )

        st.markdown("### 📄Summary")
        summary = {
            "Avg Retrieval (Q–Doc)": bench_df["avg_q_doc_sim"].mean(),
            "Avg Groundedness": bench_df["answer_context_sim"].mean(),
            "Avg Relevance": bench_df["answer_relevance_sim"].mean(),
            "Avg Coverage": bench_df["coverage"].mean(),
            "Precision Proxy": bench_df["context_precision_proxy"].mean(),
            "Recall Proxy": bench_df["context_recall_proxy"].mean(),
            "Semantic Accuracy (if GT)": bench_df["semantic_accuracy_vs_gt"].mean(skipna=True),
            "Recall@k (if label)": bench_df["retrieval_recall_at_k_label"].mean(skipna=True),
        }
        st.json({k: (float(v) if pd.notna(v) else None) for k, v in summary.items()})



Overwriting app_eval.py


In [27]:
!ls -l

total 36
-rw-r--r--  1 root root 18585 Aug 21 12:41 app_eval.py
-rw-r--r--  1 root root  6780 Aug 21 12:41 app.py
drwxr-xr-x 10 root root  4096 Aug 21 12:29 chroma_db
drwxr-xr-x  1 root root  4096 Aug 19 13:38 sample_data


## Streamlit Code for both user and developer

In [28]:
!pkill -f streamlit || echo "No old Streamlit process"
!pkill -f ngrok || echo "No old ngrok process"

^C
^C


In [29]:
import os, time, subprocess
from pyngrok import ngrok
from google.colab import userdata

# Secrets -> env
os.environ['MISTRAL_API_KEY'] = userdata.get('MISTRAL_API_KEY') or ""
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN') or ""

if not os.environ['MISTRAL_API_KEY']:
    print("❗ MISTRAL_API_KEY missing in Colab secrets.")
if not NGROK_AUTH_TOKEN:
    print("❗ NGROK_AUTH_TOKEN missing in Colab secrets.")

if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

APP_FILE = "app_eval.py"

# Launch Streamlit
process = subprocess.Popen(["streamlit", "run", APP_FILE, "--server.port=8501"])
print("Starting Streamlit server...")
time.sleep(5)

public_url = ngrok.connect(8501)
print(f"✅ {APP_FILE} is live at: {public_url}")

Starting Streamlit server...
✅ app_eval.py is live at: NgrokTunnel: "https://07d7311c9a33.ngrok-free.app" -> "http://localhost:8501"


In [30]:
import os, time, subprocess
from pyngrok import ngrok
from google.colab import userdata

# Secrets -> env
os.environ['MISTRAL_API_KEY'] = userdata.get('MISTRAL_API_KEY') or ""
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN') or ""

if not os.environ['MISTRAL_API_KEY']:
    print("❗ MISTRAL_API_KEY missing in Colab secrets.")
if not NGROK_AUTH_TOKEN:
    print("❗ NGROK_AUTH_TOKEN missing in Colab secrets.")

if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

APP_FILE = "app.py"

# Launch Streamlit
process = subprocess.Popen(["streamlit", "run", APP_FILE, "--server.port=8501"])
print("Starting Streamlit server...")
time.sleep(5)

public_url = ngrok.connect(8501)
print(f"✅ {APP_FILE} is live at: {public_url}")

Starting Streamlit server...
✅ app.py is live at: NgrokTunnel: "https://a1bcfaa2e0e7.ngrok-free.app" -> "http://localhost:8501"
